In [1]:
# Importing the required Packages
import os 
import pandas as pd
import json
# LangChain related Packages 
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI

# For OpenAPI 
from helper import get_openai_api_key

# For Visualization and User Interface 
import streamlit as st


In [2]:
# Setup the OpenAI Chat Model 
oai_model = ChatOpenAI(
    api_key=get_openai_api_key()
)

In [3]:
# Loading the CSV Data 
df = pd.read_csv("aviation_grievance_may19_2025.csv").fillna(value = 0)

In [26]:
# Prepare the LangChain Dataframe Agent 
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm=oai_model,df=df,verbose=True, allow_dangerous_code=True)


In [46]:
# Creating a Prompt 
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- If the query requires a table, format your answer like this:
           {"table": {"columns": ["column1", "column2", ...], "data": [[value1, value2, ...], [value1, value2, ...], ...]}}

- For a bar chart, respond like this:
           {"bar": {"columns": ["A", "B", "C", ...], "data": [25, 24, 10, ...]}}

- If a line chart is more appropriate, your reply should look like this:
           {"line": {"columns": ["A", "B", "C", ...], "data": [25, 24, 10, ...]}}
- If the answer is not known or available, respond with:
           {"answer": "I do not know."}

- Return all output as a string. Remember to encase all strings in the "columns" list and data list in double quotes. 
  For example: {"columns": ["Products", "Orders"], "data": [["51993Masc", 191], ["49631Foun", 152]]}

- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
"""

# QUESTION = "How many grievances were raised under each category ?" 


# agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)

In [29]:
# QUESTION = "Show me the names of the columns present"
# agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)

In [47]:
QUESTION = "show the grievances grouped by type. Use type as the column and totalReceived as the data"
response = agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...
Thought: To answer the question, I need to first set the display options to show all columns in the dataframe, then group the grievances by type and show the totalReceived for each type.
Action: python_repl_ast
Action Input: pd.set_option('display.max_columns', None)Now that all columns are displayed, I can group the grievances by type and show the totalReceived for each type.
Action: python_repl_ast
Action Input: df.groupby('type')['totalReceived'].sum()type
0                                       598
Air Charter Services                      1
Air Conditioning / Maintenance          312
Air Worthiness                           69
Aman Aviation &amp; Aerospace Sol.        1
Arrival                                 252
Aryan Aviation                            1
Baggage                                8808
Baggage / Lost & Found                  613
Baggage / Lost &amp; Found              693
Bharat Hotels Ltd                         1
Boarding Gat

In [48]:
# Handling the Response received from the Agent 
# Check if the response is a bar chart.
response_dict = json.loads(response['output'])
print(type(response_dict))
if "bar" in response_dict:
    print("Result is a bar chart")
    data = response_dict["bar"]
    try:
        df_data = {
                col: [x[i] if isinstance(x, list) else x for x in data['data']]
                for i, col in enumerate(data['columns'])
            }       
        df = pd.DataFrame(df_data)
        df.set_index("Grievances", inplace=True)
        st.bar_chart(df)
    except ValueError:
        print(f"Couldn't create DataFrame from data: {data}")

# Check if the response is a line chart.
if "line" in response_dict:
    print("Result is line chart")
    data = response_dict["line"]
    try:
        df_data = {col: [x[i] for x in data['data']] for i, col in enumerate(data['columns'])}
        df = pd.DataFrame(df_data)
        df.set_index("Products", inplace=True)
        st.line_chart(df)
    except ValueError:
        print(f"Couldn't create DataFrame from data: {data}")

# Check if the response is a table.
if "table" in response_dict:
    print("Result is a Table")
    data = response_dict["table"]
    df = pd.DataFrame(data["data"], columns=data["columns"])
    st.table(df)



2025-05-22 15:43:09.463 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-22 15:43:09.557 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-22 15:43:09.605 
  command:

    streamlit run /home/aditya/DevWork/blog_posts/dbAgents/.venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-05-22 15:43:09.605 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


<class 'dict'>
Result is a Table


In [44]:
response_dict

'```json\n{"table": {"columns": ["type", "totalReceived"], "data": [["0", 598], ["Air Charter Services", 1], ["Air Conditioning / Maintenance", 312], ["Air Worthiness", 69], ["Aman Aviation & Aerospace Sol.", 1], ["Arrival", 252], ["Aryan Aviation", 1], ["Baggage", 8808], ["Baggage / Lost & Found", 693], ["Bharat Hotels Ltd", 1], ["Boarding Gate", 40], ["Check-in & Boarding", 8185], ["Cleanliness", 235], ["Cleanliness / Hygiene", 721], ["Departure", 234], ["Engineering Training School", 8], ["Entry Passes", 17], ["Examination", 65], ["Flight Delays", 15458], ["Flying School", 1], ["Global Vectra Helicorp", 3], ["Heritage Aviation", 1], ["Himalayan Heli Services", 8], ["License", 20], ["MRO", 4], ["Maintenance", 826], ["Manufacturer", 3], ["Meal", 1260], ["Other", 10871], ["Pawan Hans Helicopters", 5], ["Permits & Permissions", 167], ["Person With Disability", 485], ["Retail Shops / Food & Beverage", 654], ["Seating / Waiting Area", 693], ["Security Check", 950], ["Security Clearances",

In [34]:
response_dict['output']

'{"table": {"columns": ["type", "totalReceived"], "data": [["0", 598], ["Air Charter Services", 1], ["Air Conditioning / Maintenance", 312], ["Air Worthiness", 69], ["Aman Aviation & Aerospace Sol.", 1], ["Arrival", 252], ["Aryan Aviation", 1], ["Baggage", 8808], ["Baggage / Lost & Found", 613], ["Baggage / Lost &amp; Found", 693], ["Bharat Hotels Ltd", 1], ["Boarding Gate", 40], ["Check-in & Boarding", 3642], ["Check-in &amp; Boarding", 4543], ["Cleanliness", 235], ["Cleanliness / Hygiene", 721], ["Departure", 234], ["Engineering Training School", 8], ["Entry Passes", 17], ["Examination", 65], ["Flight Delays", 15458], ["Flying School", 1], ["Global Vectra Helicorp", 3], ["Heritage Aviation", 1], ["Himalayan Heli Services", 8], ["License", 20], ["MRO", 4], ["Maintenance", 826], ["Manufacturer", 3], ["Meal", 1260], ["Other", 10871], ["Pawan Hans Helicopters", 5], ["Permits & Permissions", 139], ["Permits &amp; Permissions", 28], ["Person With Disability", 485], ["Retail Shops / Food & 

In [4]:
df['subcategory'].unique()

array(['Air Canada', 'Air Costa', 'GoAir', 'Air India', 'Lufthansa',
       'Jet Airways', 'Alliance Air (India)', 'Air Arabia',
       'Thai Airways', 'IndiGo', 'Air Mauritius', 'Srilankan Airways',
       'Akasa Air', 'DGCA', 'Surat', 'Hyderabad', 'Thiruvananthapuram',
       'Delhi', 'Varanasi', 'Bhubaneswar', 'Malda', 'Aurangabad',
       'Mumbai', 'Visakhapatnam', 'Patna', 'Gorakhpur', 'Ahmedabad',
       'Bengaluru', 'Indore', 'Pune', 'Chennai', 'Jammu', 'Goa',
       'Kolkata', 'Aeroflot', 'Emirates Airline', 'Kuwait Airways',
       'Qatar Airways', 'Air Asia Berhad (Intl Tickets)',
       'Ethiopian Airlines', 'Tiger Airways', 'Star Air',
       'Etihad Airways', 'Silk Air', 'Air France', 'FlyBig',
       'Cathay Pacific', 'Singapore Airlines', 'Turkish Airlines',
       'Egypt Air', 'Oman Airlines', 'Saudia', 'Air Astana',
       'Air Seychelles', 'Tiruchirappalli', 'Madurai', 'Nagpur',
       'Bagdogra', 'Kochi', 'Amritsar', 'Jodhpur', 'Jaipur', 'Ranchi',
       'Chandigarh'